# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,axim,4.8699,-2.2405,80.51,80,41,7.92,GH,1733685863
1,1,solsona,41.9940,1.5171,39.00,72,100,5.99,ES,1733685865
2,2,crane,31.3974,-102.3501,68.68,22,1,23.29,US,1733685596
3,3,saint-philippe,-21.3585,55.7679,77.16,85,18,10.27,RE,1733685867
4,4,olinda,-8.0089,-34.8553,83.93,70,75,13.80,BR,1733685868


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Date Cleaning w/dates
city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,axim,4.8699,-2.2405,80.51,80,41,7.92,GH,2024-12-08 19:24:23
1,1,solsona,41.9940,1.5171,39.00,72,100,5.99,ES,2024-12-08 19:24:25
2,2,crane,31.3974,-102.3501,68.68,22,1,23.29,US,2024-12-08 19:19:56
3,3,saint-philippe,-21.3585,55.7679,77.16,85,18,10.27,RE,2024-12-08 19:24:27
4,4,olinda,-8.0089,-34.8553,83.93,70,75,13.80,BR,2024-12-08 19:24:28


In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 75
max_temp = 85
max_wind = 15

# Drop any rows with null values
city_data_df2 = city_data_df.dropna()

mask = (city_data_df2["Max Temp"] >= min_temp) & (city_data_df2["Max Temp"] < max_temp) & (city_data_df2["Wind Speed"] < max_wind)
city_data_df2 = city_data_df2.loc[mask].reset_index(drop=True)

# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,axim,4.8699,-2.2405,80.51,80,41,7.92,GH,2024-12-08 19:24:23
1,3,saint-philippe,-21.3585,55.7679,77.16,85,18,10.27,RE,2024-12-08 19:24:27
2,4,olinda,-8.0089,-34.8553,83.93,70,75,13.80,BR,2024-12-08 19:24:28
3,9,bargaal,11.2864,51.0773,75.97,77,21,10.51,SO,2024-12-08 19:24:35
4,22,richards bay,-28.7830,32.0377,76.30,85,43,9.35,ZA,2024-12-08 19:24:52
...,...,...,...,...,...,...,...,...,...,...
122,548,humberto de campos,-2.5983,-43.4611,84.79,60,72,8.12,BR,2024-12-08 19:35:22
123,550,obo,5.3998,26.4869,77.61,32,99,4.88,CF,2024-12-08 19:35:24
124,555,bontang,0.1333,117.5000,77.43,85,94,3.22,ID,2024-12-08 19:35:30
125,557,kotido,2.9806,34.1331,75.16,40,8,6.44,UG,2024-12-08 19:35:32


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
hotel_rows = []

for index, row in city_data_df2.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df2)
hotel_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,axim,4.8699,-2.2405,80.51,80,41,7.92,GH,2024-12-08 19:24:23
1,3,saint-philippe,-21.3585,55.7679,77.16,85,18,10.27,RE,2024-12-08 19:24:27
2,4,olinda,-8.0089,-34.8553,83.93,70,75,13.80,BR,2024-12-08 19:24:28
3,9,bargaal,11.2864,51.0773,75.97,77,21,10.51,SO,2024-12-08 19:24:35
4,22,richards bay,-28.7830,32.0377,76.30,85,43,9.35,ZA,2024-12-08 19:24:52
...,...,...,...,...,...,...,...,...,...,...
122,548,humberto de campos,-2.5983,-43.4611,84.79,60,72,8.12,BR,2024-12-08 19:35:22
123,550,obo,5.3998,26.4869,77.61,32,99,4.88,CF,2024-12-08 19:35:24
124,555,bontang,0.1333,117.5000,77.43,85,94,3.22,ID,2024-12-08 19:35:30
125,557,kotido,2.9806,34.1331,75.16,40,8,6.44,UG,2024-12-08 19:35:32


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df2)
categories = "hotel.name"
radius = 10000 #10,000m

filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 5

params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    [hotel_df.Lat, 
     hotel_df.Lng]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = filters
    params["bias"] = bias

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
        # print(response.status_code)
    data = name_address.json()

    name_address = response.json()
        # Print the results
    results = data.get("features", [])
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
axim - nearest hotel: No hotel found
richards bay - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
saint-joseph - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
takum - nearest hotel: No hotel found
tanjung pandan - nearest hotel: No hotel found
luanda - nearest hotel: No hotel found
bikenibeu village - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
guaraciaba do norte - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
nova sintra - nearest hotel: No hotel found
porto novo - nearest hotel: No hotel found
south hedland - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
savanna-la-mar - nearest hotel: No hotel found
port blair - nearest hotel: No hotel found
pemba - 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,axim,4.8699,-2.2405,80.51,80,41,7.92,GH,2024-12-08 19:24:23,No hotel found
1,22,richards bay,-28.7830,32.0377,76.30,85,43,9.35,ZA,2024-12-08 19:24:52,No hotel found
2,25,avarua,-21.2078,-159.7750,78.85,78,100,2.30,CK,2024-12-08 19:24:55,No hotel found
3,29,saipan,15.1355,145.7010,81.30,84,40,5.75,MP,2024-12-08 19:25:00,No hotel found
4,30,la'ie,21.6477,-157.9253,79.05,89,2,8.90,US,2024-12-08 19:25:01,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
90,548,humberto de campos,-2.5983,-43.4611,84.79,60,72,8.12,BR,2024-12-08 19:35:22,No hotel found
91,550,obo,5.3998,26.4869,77.61,32,99,4.88,CF,2024-12-08 19:35:24,No hotel found
92,555,bontang,0.1333,117.5000,77.43,85,94,3.22,ID,2024-12-08 19:35:30,No hotel found
93,557,kotido,2.9806,34.1331,75.16,40,8,6.44,UG,2024-12-08 19:35:32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df2.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Hotel Name"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)